In [180]:
from AlmaIndicator import ALMAIndicator
import matplotlib.pyplot as plt
from scipy import signal
from numpy import diff
import pandas as pd
import numpy as np 

In [71]:
pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [211]:
INPUT_PATH = '/home/araz/Documents/ai/files/XAUUSD_candlestick_1D.json' 

In [272]:
df = pd.read_json(INPUT_PATH, convert_dates=True)
train_length = int(len(df) * 0.6)
df_cross_test  = df[train_length - 15:]
cross_test_length = len(df_cross_test) 
df_cross_val = df_cross_test[:cross_test_length]
df_test = df_cross_test[cross_test_length:]

In [257]:
def feature_add(df: pd.DataFrame) -> df:
    alma_indicator = ALMAIndicator(close=df['Close'])
    alma = alma_indicator.alma()
    df['alma'] = alma
    diff_price = diff(price)
    diff_price = np.insert(diff_price, 0, None)
    print(diff_price.dtype)
    df['Price Change'] = diff_price
    period_length = 14
    df['Gain'] = np.where(df['Price Change'] > 0, df['Price Change'], 0)
    df['Loss'] = np.where(df['Price Change'] < 0, abs(df['Price Change']), 0)
    df['Avg Gain'] = df['Gain'].rolling(window=period_length).mean()
    df['Avg Loss'] = df['Loss'].rolling(window=period_length).mean()
    df['RS'] = df['Avg Gain'] / df['Avg Loss']
    df['RSI'] = 100 - (100 / (1 + df['RS']))
    df = df.drop(['Avg Gain'], axis=1)
    df = df.drop(['Gain'], axis=1)
    df = df.drop(['Loss'], axis=1)
    df = df.drop(['Avg Loss'], axis=1)
    df = df.drop(df[df['RS'] == 'inf'].index)
    df['12-day EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['26-day EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD Line'] = df['12-day EMA'] - df['26-day EMA']
    df['Signal Line'] = df['MACD Line'].ewm(span=9, adjust=False).mean()
    df['MACD Histogram'] = df['MACD Line'] - df['Signal Line']
    df.dropna(axis=1, inplace=True)
    return df

In [273]:
df = feature_add(df)

float64


In [259]:
def ploting(date_list,price, tag_list):
    colors = []
    for index in tag_list:
        if index == 'B':
            colors.append('green')
        elif index == 'S':
            colors.append('red')
        elif index == 'S/B':
            colors.append('orange')
        else:
            colors.append('black')
    fig, ax = plt.subplots()
    ax.plot(date_list, price)
    ax.scatter(date_list, price, c=colors)
    ax.set_xticklabels(date_list, rotation=90, ha='right')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('total labeling')
    plt.show()

In [260]:
def top_max_peaks(smooth_peaks: np.array, y_axis: np.array) -> np.array:
    y_axis = list(y_axis)
    start_point = 0
    max_points = []
    for index in smooth_peaks:
        current_list = y_axis[start_point:index + 1]
        max_point = max(current_list)
        max_index = current_list.index(max_point) + start_point
        max_points.append(max_index)
        start_point = index
    np_max = np.array(max_points)
    return np_max


In [261]:
def top_min_peaks(smooth_peaks: np.array, y_axis: np.array) -> np.array:
    y_axis = list(y_axis)
    start_point = 0
    min_points = []
    for index in smooth_peaks:
        current_list = y_axis[start_point:index + 1]
        min_point = min(current_list)
        min_index = current_list.index(min_point) + start_point
        min_points.append(min_index)
        start_point = index
    np_min = np.array(min_points)
    return np_min



In [262]:
def plot_data_frame(x_axis, y_axis):
    list = signal.savgol_filter(y_axis,21, 3)
    smooth_peaks, _ = signal.find_peaks(x=list)
    main_peaks, _ = signal.find_peaks(x=y_axis)
    negated_list = [-x for x in y_axis]
    negated_smooth_list = [-x for x in list]
    main_valley, _ = signal.find_peaks(x=negated_list)
    min_smooth_peaks, _ = signal.find_peaks(x=negated_smooth_list)
    max_points = top_max_peaks(smooth_peaks, y_axis)
    min_points = top_min_peaks(min_smooth_peaks, y_axis)
    tag_list = []
    red_list = []
    green_list = []
    flag = True
    signal_list = []
    buy_price = None
    buy = None
    row = 0
    just_buy = 0
    just_sell = 0
    for i in range(len(x_axis)):
        if i in min_points:
            flag = True
            red_list.append(i)
            tag_list.append('B')
            signal_list.append('B')
            buy = x_axis[i]
            buy_price = y_axis[i]
            just_buy += 1
        elif i in max_points:
            green_list.append(i)
            signal_list.append('S')
            tag_list.append('S')
            just_sell += 1
            if buy != None:
                row += 1
            flag = False

        else:
            if flag == True:
                red_list.append(i)
                try:
                    risk_reward = (0.95 * buy_price) / (y_axis[i] - buy_price)
                    if risk_reward < 1.5:
                        signal_list.append('B')
                        tag_list.append('B')
                        just_buy += 1
                    else:
                        signal_list.append('B')
                        tag_list.append('B')
                        buy_price = y_axis[i]
                        row += 1
                        buy = x_axis[i]
                        just_buy += 1
                except:
                    signal_list.append('B')
                    tag_list.append('B')
                    just_buy += 1
            else:
                green_list.append(i)
                signal_list.append('S')
                tag_list.append('S')
                just_sell += 1
    # ploting(x_axis, y_axis, tag_list)
    return tag_list, signal_list

In [274]:
price = df['Close']
date_array = np.array(list(map(str, df['Open Time'])))
price_array = np.array(price)
tag_list, signal_list = plot_data_frame(date_array,price) 
df.dropna(axis=1, inplace=True)
df['tag'] = tag_list
df.dropna(axis=1, inplace=True)
print(df)
y = np.array(df['tag'])
x = np.drop(['id', 'Open Time', 'tag'], axis='columns') 
print(df)

        id            Open Time     Open     High      Low    Close  \
0        0  2013-01-01-21-00-00  1681.18  1694.82  1680.61  1686.53   
1        1  2013-01-02-21-00-00  1685.55  1689.95  1660.75  1663.67   
2        2  2013-01-03-21-00-00  1663.24  1664.05  1625.76  1655.46   
3        3  2013-01-06-21-00-00  1656.11  1662.45  1642.76  1646.77   
4        4  2013-01-07-21-00-00  1647.36  1662.52  1646.55  1658.98   
...    ...                  ...      ...      ...      ...      ...   
2723  2723  2023-07-19-21-00-00  1976.32  1987.44  1965.36  1969.46   
2724  2724  2023-07-20-21-00-00  1970.14  1973.77  1956.82  1961.78   
2725  2725  2023-07-23-21-00-00  1962.09  1967.86  1953.45  1954.78   
2726  2726  2023-07-24-21-00-00  1955.09  1965.67  1951.55  1964.87   
2727  2727  2023-07-25-21-00-00  1964.93  1974.19  1961.99  1971.81   

       12-day EMA   26-day EMA  MACD Line  Signal Line  MACD Histogram tag  
0     1686.530000  1686.530000   0.000000     0.000000        0.000000